# Importes

In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import slate3k as slate

# Funções úteis

In [44]:
def requestSoup(url):
    response = get(url)
    return BeautifulSoup(response.text, 'html.parser')

def getIntervalos(posi,listaPrincipal):
    intervalos = []
    for i in range(len(posi)):
        if(i != len(posi)-1):
            intervalos.append((posi[i]+1,posi[i+1]))
        else:
            intervalos.append((posi[i]+1,len(listaPrincipal)))
    return intervalos

def getPerguntas(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i].text):
            if(chaves is not None):
                texto = soup[i].text
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i].text)
                posi.append(i)
                
    return [perguntas,posi]
                
def getPerguntasPDF(soup, chaves = None):
    perguntas = []
    posi = []
    for i in range(len(soup)):
        if("?" in soup[i]):
            if(chaves is not None):
                texto = soup[i]
                for j in chaves:
                    texto = texto.replace(j,"")
                perguntas.append(texto)
                posi.append(i)
            else:
                perguntas.append(soup[i])
                posi.append(i)
    return [perguntas,posi]

def getRespostasPDF(intervalos,soup):
    respostas = []
    for i in intervalos:
        text = ""
        for j in range(i[0],i[1]):
            text += soup[j]
        respostas.append(text)
    return respostas

def getComparacao(first,second):
    if(len(first) == len(second)):
        return True
    else:
        return False
        
def criarDataframe(perguntas,respostas):
    return pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas})

# Código

In [11]:
url = "https://www.unasus.gov.br/noticia/oms-perguntas-e-respostas-sobre-covid-19-gravidez-parto-e-amamentacao"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [12]:
html_soup = html_soup.find_all(id = 'noticia_texto')

In [18]:
html_p = html_soup[0].find_all('p')

In [30]:
perguntas = []
posi = []
a = 0
for i in html_p:
    if("?" in i.text):
        perguntas.append(i.text.replace("\xa0",""))
        posi.append(a)
    
    a += 1
        
print(perguntas)

['As gestantes correm mais risco de contrair a COVID-19?', 'Estou grávida. Como posso me proteger contra a COVID-19?', 'As gestantes devem ser testadas em relação à COVID-19?', 'A COVID-19 pode ser transmitida da mulher para seu bebê ainda por nascer ou recém-nascido?', 'Que cuidados devem ser tomados durante a gestação e o parto?', 'As gestantes com suspeita ou confirmação de infecção pela COVID-19 devem necessariamente serem submetidas ao parto cesariano?', 'As mulheres infectadas pela COVID-19 podem amamentar?', 'Posso tocar e segurar meu bebê se tenho COVID-19?', 'Tenho COVID-19 e me sinto muito debilitada para amamentar meu bebê diretamente. O que posso fazer?']


In [40]:
intervalos = []
for i in range(len(posi)):
    if(i == len(posi)-1):
        intervalos.append((posi[i], len(html_p)))
    else:
        intervalos.append((posi[i], posi[i + 1]))

In [41]:
intervalos

[(1, 5),
 (5, 9),
 (9, 12),
 (12, 14),
 (14, 18),
 (18, 21),
 (21, 24),
 (24, 27),
 (27, 30)]

In [44]:
respostas = []
for i in intervalos:
    text = ""
    for j in range(i[0] + 1, i[1]):
        text += html_p[j].text
    respostas.append(text)

In [46]:
len(perguntas) == len(respostas)

True

In [52]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv("./UNA-SUS.csv", index = False)

# Código 2

In [69]:
with open('./img/3.pdf', 'rb') as f:
        extracted_text = slate.PDF(f)
extracted_text = extracted_text[0].replace("\n\n","").replace("\x0c","").replace("SUMÁRIO","").split("\n")
extracted_text

['1. Quais são os EPIs que devo utilizar?............................................................05',
 '2. Qual a diferença da máscara cirúrgica, máscara N95 e Face Shield? ',
 'Quando e por quanto tempo devo usar cada uma?..........................................07',
 '3. O profissional de enfermagem pode utilizar máscara caseira em ',
 'serviços de saúde?.............................................................................................................09',
 '4. Qual é a ordem correta de paramentação e desparamentação?..........10 ',
 '5. O que fazer quando faltar capote?........................................................................11 ',
 '6. Se faltar o EPI adequado, o que devo fazer? Posso me recusar a ',
 'prestar assistência?..........................................................................................................11',
 '7. Técnico em enfermagem também faz e dá o diagnóstico do teste ',
 'rápido?..............................................

In [70]:
posi = []
for i in range(len(extracted_text)):
    try:
        if(int(extracted_text[i][0])):
            posi.append(i)
    except:
        pass

In [71]:
posi

[0, 1, 3, 5, 6, 7, 9, 11, 12, 14, 16, 17, 18, 21, 22]

In [72]:
intervalos = []
for i in range(len(posi)):
    if(i == len(posi)-1):
        intervalos.append((posi[i], len(extracted_text)))
    else:
        intervalos.append((posi[i], posi[i + 1]))

In [73]:
intervalos

[(0, 1),
 (1, 3),
 (3, 5),
 (5, 6),
 (6, 7),
 (7, 9),
 (9, 11),
 (11, 12),
 (12, 14),
 (14, 16),
 (16, 17),
 (17, 18),
 (18, 21),
 (21, 22),
 (22, 24)]

In [74]:
perguntas = []
for i in intervalos:
    text = ""
    for j in range(i[0], i[1]):
        text += extracted_text[j]
    text = text[3:].split("..")
    perguntas.append((text[0]))

In [49]:
respostas = []
for i in perguntas:
    with open(f'./img/{i[1]-1}.pdf', 'rb') as f:
        extracted_text = slate.PDF(f)
        respostas.append(extracted_text[0])

In [75]:
respostas = ["A Agência Nacional de Vigilância Sanitária (Anvisa), a Organização Mundi-al da Saúde (OMS) e o Conselho Federal de Enfermagem (Cofen) elaboraram recomendações  baseadas  nas  evidências  científicas  até  então  produzidas  sobre a COVID-19. Todas as entidades apontam que a transmissão do novo coronavírus  (SARS-CoV-2)  ocorre  por  meio  de  gotículas  respiratórias,  por  contato  (direto  ou  indireto)  e  pelo  contato  com  os  aerossóis  do  paciente  contaminado. Sendo assim, as precauções a serem adotadas pelos serviços de saúde referem-se a transmissão por gotículas e aerossóis e por contato. Em termos de EPI a serem utilizados por profissionais da saúde em quar-to/área/enfermaria/box  de  pacientes  suspeitos  ou  confirmados  de  COV-ID-19,  recomenda-se  o  uso  de  óculos  de  proteção  ou  protetor  facial  (face  shield),  máscara  N95/PFF2,  avental  e  luvas  de  procedimento,  além  da  higiene das mãos. Em áreas coletivas em que há procedimentos geradores de  aerossóis  é  necessário  a  avaliação  de  risco  quanto  a  indicação  do  uso  máscara  N95/PFF2  ou  equivalente  pelos  outros  profissionais  dessa  área,  que não estão envolvidos diretamente com esse procedimento. Quando o profissional atuar em procedimentos com risco de geração de aerossóis,  em  pacientes  suspeitos  ou  confirmados  de  infecção  pelo  novo  coronavírus,  deve  utilizar  a  máscara  de  proteção  respiratória  (respirador  particulado) com eficácia mínima na filtração de 95% de partículas de até 0,3μ (tipo N95, N99, N100, PFF2 ou PFF3). São alguns exemplos de procedi-mentos com risco de geração de aerossóis: intubação ou aspiração traqueal, ventilação  não  invasiva,  ressuscitação  cardiopulmonar,  ventilação  manual  antes da intubação, coletas de secreções nasotraqueais, broncoscopias, etc.É  importante  ressaltar  que  a  máscara  N95/PFF2  ou  equivalente  com  válvula  expiratória  não  pode  ser  utilizada  como  controle  de  fonte,  pois  ela  permite a saída do ar expirado pelo profissional que, caso esteja infectado, poderá  contaminar  pacientes,  outros  profissionais  e  o  ambiente.  Caso  só  tenha   disponível   este   modelo   de   máscara   com   válvula   expiratória,   recomenda-se o uso concomitante de um protetor facial.Quanto  às  áreas  de  assistência  a  pacientes  (por  exemplo,  enfermarias,  quartos, consultório) não confirmados de COVID-19, recomenda-se a máscara  cirúrgica  (+  outros  EPIs  de  acordo  com  a  precaução  padrão  e,  se  necessário, precauções específicas). É importante se atentar para as outras medidas de prevenção, como higiene das mãos e distância de pelo menos 1 metro de outras pessoas.O avental não precisa ser necessariamente impermeável, mas o profis-sional de saúde deve avaliar cada situação e deve utilizar o avental imper-meável dependendo do quadro clínico do paciente (vômitos, diarreia, hiper-secreção  orotraqueal,  sangramento,  dentre  outros).  A  gramatura  indicada  para  o  avental  ou  capote  utilizado  em  serviços  de  saúde  é  de  no  mínimo  30g/m2 e, no caso de ser impermeável, a gramatura mínima deve ser de 50 g/m2.  As  luvas  de  procedimentos  não  cirúrgicos  devem  ser  utilizadas,  no  contexto da epidemia da COVID-19, em qualquer contato com o paciente ou seu entorno (precaução de contato). Quando o procedimento a ser realizado no paciente exigir técnica asséptica, devem ser utilizadas luvas estéreis (de procedimento cirúrgico).","A  principal  diferença  entre  a  máscara  cirúrgica  e  o  protetor  respiratório  N95 ou similar, é o tipo e tamanho de partícula que filtram, proporcionado pelo material usado no sua fabricação e especificações técnicas.As máscaras cirúrgicas devem ser utilizadas para evitar a contaminação da boca e nariz do profissional por gotículas respiratórias, quando o mesmo atuar a uma distância inferior a 1 metro do paciente suspeito ou confirmado de  infecção  pelo  novo  coronavírus.  A  máscara  deve  ser  confeccionada  de  material tecido-não tecido (TNT), possuir no mínimo uma camada interna e uma camada externa e obrigatoriamente um elemento filtrante. A camada externa  e  o  elemento  filtrante  devem  ser  resistentes  à  penetração  de  fluidos transportados pelo ar (repelência a fluidos). Deve ser confeccionada de forma a cobrir adequadamente a área do nariz e da boca do usuário, pos-suir  um  clipe  nasal  constituído  de  material  maleável  que  permita  o  ajuste  adequado do contorno do nariz e das bochechas. E o elemento filtrante deve possuir  eficiência  de  filtragem  de  partículas  (EFP)  >  98%  e  eficiência  de  filtragem bacteriológica (BFE) > 95%.A máscara  de  proteção  respiratória  (respirador  particulado)  N95  é  um  dispositivo com eficácia mínima na filtração de 95% de partículas de até 0,3μ, semelhante aos tipos N99, N100, PFF2 ou PFF3. É um equipamento de proteção  individual  (EPI)  que  cobre  a  boca  e  o  nariz,  proporciona  uma  vedação  adequada  sobre  a  face  do  usuário,  possui  filtro  eficiente  para  retenção   dos   contaminantes   atmosféricos   presentes   no   ambiente   de trabalho na forma de aerossóis.O  protetor  ocular  ou  protetor  de  face  (FACE  SHIELD),  são  óculos  de  proteção ou protetores faciais que cobrem a frente e os lados do rosto, os quais devem ser utilizados quando houver risco de exposição do profissional a  respingos  de  sangue,  secreções  corporais  e  excreções.  Os  óculos  de  proteção  ou  protetores  faciais  devem  ser  exclusivos  de  cada  profissional  responsável pela assistência, devendo após o uso sofrer limpeza e posterior desinfecção com álcool líquido a 70%, hipoclorito de sódio ou outro desinfe-tante  recomendado  pelo  fabricante.  Caso  o  protetor  facial  tenha  sujidade  visível,  deve  ser  lavado  com  água  e  sabão/detergente  e  só  depois  dessa  limpeza, passar pelo processo de desinfecção.","As máscaras de tecido não são consideradas EPI porque não atendem a Resolução  RDC  Nº  379,  DE  30  DE  ABRIL  DE  2020.  Sendo  assim  servem  para impedir que a pessoa que a está usando espalhe secreções respiratóri-as ao falar, espirrar ou tossir (controle da fonte), desde que estejam limpas e secas,  portanto  não  devem  ser  usadas  por  profissionais  que  atuam  nos  serviços   de   saúde,   conforme   descrito   na   Nota   Técnica   Nº   04/2020   GVIMS/GGTES/Anvisa  e  nas  Orientações  Sobre  a  Colocação  Dos  Equipa-mentos De Proteção Individual (EPI) do Conselho Federal de Enfermagem.Quem pode usar máscaras de tecido dentro dos serviços de saúde, con-forme especificado na Norma Técnica da Anvisa? Pacientes assintomáticos, visitantes  e  acompanhantes,  profissionais  que  atuam  na  recepção,  áreas  administrativas  (quando  não  tiver  contato  a  menos  de  1  metro  com  pacientes),  profissionais  de  áreas  em  que  não  há  assistência  a  pacientes  como manutenção, almoxarifado, farmácia, etc (quando não tiver contato a menos  de  1  metro  com  pacientes),  profissionais  de  saúde  e  de  apoio  em  situações  em  que  não  há  necessidade  do  uso  de  máscara  cirúrgica  ou  de  máscara de proteção respiratória N95/PFF2.", "Antes  de  iniciar  a  paramentação  e  após  a  paramentação,  lave  as  mãos  com água e sabão ou higienize com solução alcoólica a 70%.PARAMENTAÇÃO1º Avental ou capote2º Máscara cirúrgica ou Máscara de proteção respiratória*3º Óculos ou protetor facial4º Gorro ou touca*5º Luvas*No  caso  de  procedimentos  gera-dores de aerossóis DESPARAMENTAÇÃO1º Luvas 2º Avental ou capote3º Gorro ou touca*4º Óculos ou protetor facial*5º Máscara cirúrgica ou de proteção respiratória*","Na falta de capotes ou avental nas instituições de saúde, o profissional de enfermagem  deverá  comunicar  imediatamente  por  escrito  e  ou  por  meio  eletrônico aos responsáveis pela unidade, assim como também ao Conselho Regional de Enfermagem.","É um direito do profissional exercer a Enfermagem com liberdade, segu-rança  técnica,  científica  e  ambiental  e  com  autonomia.  Desta  forma,  o  profissional  de  enfermagem  deve  avaliar  cada  situação  da  assistência  de  enfermagem para prestar assistência segura ao paciente e para si mesmo. De acordo com o Art. 17 da Resolução RDC 63/2011, que dispõe sobre os  Requisitos  de  Boas  Práticas  de  Funcionamento  para  os  Serviços  de  Saúde,   o   serviço   de   saúde   deve   prover   infraestrutura   física,   recursos   humanos,  equipamentos,  insumos  e  materiais  necessários  à  operacional-ização  do  serviço  de  acordo  com  a  demanda,  modalidade  de  assistência  prestada e a legislação vigente.O profissional, ao constatar a ausência de EPI adequado ao atendimento que realiza (verificar quais EPI são recomendados para cada setor e tipo de atendimento  específico  conforme  Nota  Técnica  GVIMS/GGTES/Anvisa  -  04/2020),  deve  antes  procurar  os  gestores  responsáveis  pelo  seu  serviço  de saúde para comunicar o fato e solicitar o reabastecimento. Recomenda-se  que  antes  de  tomar  qualquer  decisão  de  negativa  da  continuidade da assistência, certifique-se de que realmente a desabasteci-mento  dos  EPI.  Persistindo  o  problema  do  desabastecimento,  denuncie  junto ao COREN-DF e demais órgãos de controle.O  profissional  poderá  suspender  suas  atividades  individuais,  quando  o  local de trabalho não oferecer condições seguras para o exercício profission-al e houver risco para sua integridade física, desde que, não seja uma situ-ação de urgência, emergência, epidemia, desastre e catástrofe. Desta forma, o profissional deverá formalizar imediatamente sua decisão por escrito e/ou por meio de correio eletrônico à instituição e ao Conselho Regional de Enfer-magem.", "O  enfermeiro  tem  competência  técnica  e  legal  para  a  realização  do  exame, aconselhamento pré-teste e pós- teste rápido para diagnóstico de COVID-19, emissão de laudo, realização ou solicitação de exame para con-firmação  diagnóstica,  encaminhamentos,  agendamentos  e  eventos  que  necessitem de sua supervisão ou orientação.  Os testes rápidos para COVID-19 são metodologicamente equiparáveis a  outros  testes  já  realizados  pelas  equipes  da  Atenção  Básica,  como,  por  exemplo,  o  teste  de  glicemia.  Os  testes  rápidos  devem  ser  amplamente  utilizados para triagem, sendo seu resultado reagente, não definem o diag-nóstico, o que vai requerer a realização de testes complementares e receber atendimento clínico.O  Técnico  e/ou  Auxiliar  de  Enfermagem  devidamente  treinado  e  sob  a  supervisão do enfermeiro pode realizar teste rápido para triagem do COV-ID-19,  encaminhando  prontamente  para  o  enfermeiro,  os  clientes  com  resultado reagente. Cabe-lhe a anotação em prontuário ou boletim de aten-dimento, da data e hora do procedimento, aspecto da polpa digital ou local de  punção,  desconforto  decorrente  da  perfuração  necessária,  resultados  encontrados,  incluindo  as  orientações  efetuadas,  nome  completo  e  Coren  do responsável pelo procedimento. O Técnico e/ou Auxiliar de Enfermagem não pode emitir laudo, que é priv-ativo do enfermeiro ou profissional de nível superior. Todavia, deve colaborar com o enfermeiro durante todo o procedimento, disponibilizando insumos e recursos  necessários  além  de  condições  adequadas  para  procedimento  e  acolhimento.  Ressalva-se  que  os  profissionais  necessitam  estar  devida-mente  capacitados  para  a  realização  do  procedimento  como  preconiza  a  legislação."]

In [76]:
respostas.extend(["Sim.  O  uso  de  máscaras  pelos  acompanhantes  se  faz  necessário  para  proteção  do  mesmo  e  das  outras  pessoas,  visto  que  o  acompanhante  é  considerado um caso suspeito. A máscara impedirá que seja um difusor do vírus. As orientações destacam, no entanto, que o uso da máscara por si só é insuficiente  para  oferecer  um  nível  adequado  de  proteção,  e  que  para  isso  também  devem  ser  adotadas  outras  medidas  como  a  higiene  das  mãos,  evitar contatos desnecessários e manter o distanciamento, para impedir a transmissão do novo coronavírus","O  profissional  deverá  ser  afastado  por  14  dias  como  medida  de  con-tenção da proliferação. Este deve ser acompanhado para observação de seu quadro respiratório. Quanto a equipe, todos devem ser testados e monito-rados","Primeiramente o gestor de enfermagem deve realizar o remanejamento adequado.  Em  segundo  lugar  o  gestor  deve  informar  aos  superiores  a  necessidade de reforço de quantitativo de pessoal. A gestão deve avaliar a continuidade do serviço e se possui relação com o atendimento prioritário a população no momento de pandemia, podendo o serviço ser interrompido e a equipe remanescente ser realocada em áreas com necessidade de reforço. O Responsável Técnico (RT) deve elaborar o cálculo de dimensionamento de pessoal de enfermagem conforme a Resolução Cofen 543/2017 e apre-sentar o déficit aos gestores/superiores para providências. E caso não sejam adotadas medidas para o saneamento do déficit, formalizar denúncia para o Coren-DF por meio do Fale Conosco.","Sim.  O  dimensionamento  de  pessoal  visando  atender  as  necessidades  da instituição e da população é uma atribuição do gestor de enfermagem.","Sim. Gestantes e lactantes não devem exercer trabalhos em locais insa-lubres, conforme Lei 13.467/17 que altera o artigo 394-A da Consolidação das Leis Trabalhistas (CLT). ","Para  o  atendimento  a  pacientes  em  atenção  domiciliar  com  infecção  suspeita  ou  confirmada  pelo  novo  coronavírus,  os  profissionais  de  saúde  devem atender à Nota Técnica GVIMS/GGTES/Anvisa - 04/2020.Para pacientes em atendimento domiciliar sem suspeita ou confirmação de  infecção  pelo  novo  coronavírus,  mas  que  apresentem  baixa  imunidade,  os cuidados visam evitar a exposição ao vírus (ANVISA, 2020): • Antes de entrar no quarto do paciente deve-se realizar a higiene das mãos; •  Acomodar  o  paciente  em  um  quarto  individual  bem  ventilado  (com janelas abertas); • Os membros da família com suspeita de infecção ou diagnóstico confir-mado de COVID-19 devem ficar em um quarto diferente do paciente e não deverão ter contato com o mesmo; • Os membros da família com suspeita de infecção ou diagnóstico confir-mado  de  COVID-19  devem  utilizar  máscara  cirúrgica  e  realizar  a  higiene  respiratória/etiqueta da tosse: a) o se tossir ou espirrar, cobrir o nariz e a boca com a parte de dentro do braço flexionado ou lenço de papel; b)  o  utilizar  lenço  descartável  para  higiene  nasal  (descartar  imediata-mente após o uso e realizar a higiene das mãos);  c) o evitar tocar os olhos, o nariz e a boca;  d) o higienizar as mãos frequentemente.  • Profissionais de saúde/cuidadores com suspeita de infecção ou diag-nóstico  confirmado  de  COVID-19  não  devem  prestar  assistência  a  esses  pacientes até a sua total recuperação; • As visitas devem ser restritas e não devem ser permitidos, em nenhuma  circunstância,  visitantes  com  sinais  de  infecção  respiratória  (tosse,  espirros, dificuldade para respirar, etc.); •  Familiares  ou  profissionais  de  saúde/cuidadores  devem  realizar  a higiene das mãos antes e após a preparação dos alimentos, após o uso do banheiro  e  sempre  que  as  mãos  estiverem  sujas.  Para  mãos  visivelmente  sujas  ou  contaminadas  com  sangue  ou  outros  fluidos  corporais,  deve-se  usar sabonete líquido e água; • Deve-se reforçar a limpeza e desinfecção das superfícies do quarto do paciente diariamente, principalmente aquelas que são mais tocadas.  • Os profissionais de saúde/cuidadores não devem ter contato com os familiares enfermos na casa;  • Devem estar disponíveis frascos de preparação alcoólica a 70% para a higiene das mãos no quarto do paciente; • Os profissionais que prestarem assistência ao paciente devem realizar a higiene das mãos conforme preconizado nos 5 momentos da Organização Mundial da Saúde: antes de contato com o paciente, antes da realização de procedimentos assépticos, após risco de exposição a fluidos corporais, após contato  com  o  paciente  e  após  contato  com  áreas  próximas  ao  paciente.  Sendo assim, os membros da família e profissionais de saúde/cuidadores", "Não.  Conforme  Resolução  Cofen  460/2014,  em  seu  Art.  4º,  a  CIP  é  de  uso  pessoal,  intransferível  e  obrigatória  para  o  exercício  das  atividades  profissionais de enfermagem.", "Procure  seu  Conselho  Regional  de  Enfermagem  para  regularizar.  Você  pode fazer isso pelo atendimento remoto. Acesse o link www.coren-df.gov-.br/site/renovacao-de-carteira/ e veja a documentação necessária. Durante  a  pandemia,  não  haverá  atendimento  presencial.  Então,  você  pode entrar em contato pelo 0800 702 3754 ou enviar uma mensagem via WhatsApp    para    os    telefones:    (61)    99606-8567    |    99171-8773    |        98343-9971 | 98211-4508"])

In [78]:
len(respostas) == len(perguntas)

True

In [80]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv("./coren-df.csv", index = False)

# Código 3

In [81]:
url = "http://breves.pa.gov.br/perguntas-e-respostas-sobre-o-covid-19/"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')

In [86]:
html_soup.find_all('div',class_ = 'entry-content clearfix')[0].find_all('div',class_ = 'col-md-12 espaco')

[<div class="col-md-12 espaco">
 <p>Os coronavírus são uma grande família viral, que causam infecções respiratórias em seres humanos e em animais. Em 80% dos casos, infecções por coronavírus causam doenças respiratórias leves a moderadas, semelhantes a um resfriado comum.</p>
 <p> </p>
 </div>, <div class="col-md-12 espaco">
 <p>Até o momento, não. No entanto, cientistas ao redor do mundo (inclusive no Brasil) já iniciaram pesquisas para o desenvolvimento de uma vacina. Ainda é muito cedo para indicar se e quando ela estará disponível.</p>
 <p> </p>
 </div>, <div class="col-md-12 espaco">
 <p>Os sinais e sintomas clínicos são principalmente respiratórios, semelhantes aos de um resfriado comum. Em casos mais graves, podem também causar infecção do trato respiratório inferior, como as pneumonias.</p>
 <p> </p>
 </div>, <div class="col-md-12 espaco">
 <p>O profissional de saúde deverá estar usando máscara cirúrgica, luvas, avental não estéril e óculos de proteção. Em alguns momentos eles 

In [96]:
perguntas = html_soup.find_all('div',class_ = 'entry-content clearfix')[0].find_all('div',class_ = 'titulo')

In [97]:
respostas = html_soup.find_all('div',class_ = 'entry-content clearfix')[0].find_all('div',class_ = 'col-md-12 espaco')

In [98]:
perguntas = [i.text for i in perguntas]
respostas = [i.text.replace("\n","") for i in respostas]

In [100]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv("./breves-pa.csv", index = False)

# Código 4

In [35]:
html_soup = RequestSoup("http://coronavirus.saude.mg.gov.br/blog/95-conversar-sobre-covid-19-com-criancas")

In [36]:
html_soup = html_soup.find_all('div',itemprop = 'articleBody')[0]

In [37]:
perguntas = html_soup.find_all('h2')
perguntas = [i.text for i in perguntas]
perguntas

['Como conversar sobre a covid-19 com uma criança?',
 'Como explicar para uma criança o que é a covid-19?',
 'Como manejar o estresse na criança ao falar sobre a covid-19?',
 'Como explicar para uma criança sobre a forma de contágio?',
 'Como falar com uma criança sobre os sintomas da covid-19?',
 'Como responder para uma criança o que o novo coronavírus pode causar?',
 'Como conversar com uma criança que está com medo de ser contaminada?',
 'Como falar sobre formas de prevenção com uma criança?',
 'Como explicar para uma criança o motivo dela não poder visitar parentes e amigos?',
 'Como explicar para uma criança sobre o uso de máscaras?',
 'Preciso sair de casa para trabalhar e precisei me afastar de uma criança para evitar a transmissão da doença. Como explicar isso a ela?',
 'Uma pessoa próxima da criança está gravemente contaminada pela covid-19. Como falar sobre isso?',
 'Uma criança me perguntou se pode morrer se for contaminada pelo vírus. Como devo lidar com isso?',
 'Como exp

In [38]:
soup = html_soup.find_all(['h2','p','ul'])[3:]

In [39]:
posicoes = []
for i in range(len(perguntas)):
    for j in range(len(soup)):
        if(perguntas[i] == soup[j].text):
            posicoes.append(j)
            break

In [40]:
posicoes

[0, 3, 6, 10, 12, 15, 17, 19, 22, 25, 28, 30, 33, 36, 40]

In [41]:
intervalos = []
for i in range(len(posicoes)):
    if(i != len(posicoes)-1):
        intervalos.append((posicoes[i]+1,posicoes[i+1]))
    else:
        intervalos.append((posicoes[i]+1,len(soup)-1))
        

In [42]:
intervalos

[(1, 3),
 (4, 6),
 (7, 10),
 (11, 12),
 (13, 15),
 (16, 17),
 (18, 19),
 (20, 22),
 (23, 25),
 (26, 28),
 (29, 30),
 (31, 33),
 (34, 36),
 (37, 40),
 (41, 49)]

In [43]:
respostas = []
for i in intervalos:
    text = ""
    for j in range(i[0],i[1]):
        text += soup[j].text.replace("\n","").replace("\xa0","")
    respostas.append(text)
        

In [44]:
len(perguntas) == len(respostas)

True

In [46]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./mgGov.csv', index = False)

# Código 5

In [83]:
with open('./img/16.pdf', 'rb') as f:
        extracted_text = slate.PDF(f)
extracted_text = extracted_text[0]
extracted_text.replace("\n\n","").replace("\u202f","")

'O que fazer se eu identificar indícios de irregularidade/falsificação em testes rápidos? Quaisquer indícios de irregularidade/falsificação devem ser imediatamente comunicados à Anvisa (Ouvidoria) ou à autoridade sanitária local. É importante destacar que a Anvisa estabeleceu um amplo e robusto programa de fiscalização e verificação dos produtos disponíveis no mercado, visando a avaliação da conformidade dos produtos aos requisitos mínimos de qualidade, segurança e eficácia. Como faço para comunicar um desvio de qualidade (queixa técnica), evento adverso ou denúncia anônima de um kit para diagnóstico de Covid-19? Para notificação de produtos para diagnóstico de Cov id-19, temos os seguintes links para notificação pelos profissionais:  •         Notiﬁcação de queixa técnica de produtos para diagnóstico de Covid-19 que têm número de registro, por profissionais de saúde: https://www8.anvisa.gov.br/notivisa/frmlogin.asp  •         Notiﬁcação de evento adverso de produtos para diagnóstico

In [57]:
perguntas = ["O que são testes para Covid-19? Que tipo de amostra é usada nos testes",
"O que são testes rápidos (IgM/IgG)? ",
"O que são testes RT- PCR",
"Os testes rápidos são de uso profissional? ",
"A partir de quantos dias após o início dos sintomas é recomendado fazer um teste rápido",
"Quando devo fazer os testes rápidos (IgM/IgG)?",
"Se o resultado do teste rápido (IgM/IgG) for POSITIVO isso indica que tenho Covid-19?",
"Se o resultado do teste rápido (IgM/IgG) for NEGATIVO isso indica que não tenho Covid-19?",
"Quem fez teste rápido em laboratórios não corre um alto risco de ter recebido um resultado falso positivo ou negativo? Os testes rápidos (IgM/IgG) são confiáveis?",
"Se os testes rápidos não servem para diagnóstico (confirmar ou descartar) de infecção por Covid-19, qual a sua função?",
"Como os testes são feitos?",
"Qual a metodologia do teste rápido para a Covid-19?",
"Estes dispositivos precisam ter registro na Anvisa?",
"Os testes rápidos disponíveis no mercado foram aprovados pela Anvisa?",
"Quais são os requisitos para um teste rápido ser registrado no Brasil?",
"O que são os testes in house?",
"Há pedidos de empresas que querem registrar testes para Covid-19 no Brasil?",
"E quais são os tipos de testes aprovados até o momento?",
"A Anvisa está priorizando a análise de pedidos de registro de testes para Covid-19?",
"Quem faz a avaliação desses testes?",
"Quantos testes rápidos foram avaliados até agora? Por qual instituição foram avaliados?",
"Os testes VALIDADOS por enquanto só são usados pelo Ministério da Saúde e pela rede pública? Ou tem algum sendo utilizado pela rede privada? Quem deve VALIDAR os testes rápidos usados pela rede privada?",
"Por que a Anvisa permitiu que laboratórios – e agora farmácias – realizem testes rápidos que ainda não foram validados?",
"Onde posso fazer os testes rápidos (IgM/IgG)?",
"Quem pode comercializar os testes rápidos para Covid-19?",
"As distribuidoras de produtos para saúde podem comercializar testes para Covid-19 para qualquer estabelecimento? Podem ser comercializados testes para pessoas físicas?",
"Distribuidoras de produtos para saúde podem comercializar testes para Covid-19 para serviços de saúde privados?",
"As distribuidoras de produtos para saúde podem comercializar testes para Covid-19 para empresas que não sejam da área da saúde e desejem fazer testes em seus funcionários (como do ramo da construção civil automobilísticas etc.)? ",
"Farmácias e drogarias podem comercializar testes para Covid-19?",
"Tenho uma empresa que não é da área da saúde (construção civil, naval ou outra atividade econômica). Posso realizar os testes rápidos (IgM/IgG) nos meus funcionários?",
"É possível fazer testes rápidos “em massa” em minha empresa/instituição?",
"É possível fazer testes rápidos “em massa” em grupos vulneráveis?",
"O que fazer se eu identificar indícios de irregularidade/falsificação em testes rápidos?",
"Como faço para comunicar um desvio de qualidade (queixa técnica), evento adverso ou denúncia anônima de um kit para diagnóstico de Covid-19?"]

In [87]:
perguntas

['O que são testes para Covid-19? Que tipo de amostra é usada nos testes',
 'O que são testes rápidos (IgM/IgG)? ',
 'O que são testes RT- PCR',
 'Os testes rápidos são de uso profissional? ',
 'A partir de quantos dias após o início dos sintomas é recomendado fazer um teste rápido',
 'Quando devo fazer os testes rápidos (IgM/IgG)?',
 'Se o resultado do teste rápido (IgM/IgG) for POSITIVO isso indica que tenho Covid-19?',
 'Se o resultado do teste rápido (IgM/IgG) for NEGATIVO isso indica que não tenho Covid-19?',
 'Quem fez teste rápido em laboratórios não corre um alto risco de ter recebido um resultado falso positivo ou negativo? Os testes rápidos (IgM/IgG) são confiáveis?',
 'Se os testes rápidos não servem para diagnóstico (confirmar ou descartar) de infecção por Covid-19, qual a sua função?',
 'Como os testes são feitos?',
 'Qual a metodologia do teste rápido para a Covid-19?',
 'Estes dispositivos precisam ter registro na Anvisa?',
 'Os testes rápidos disponíveis no mercado fora

In [84]:
respostas = ["Os testes para Covid-19 são produtos para diagnóstico de uso in vitro, nos termos da RDC 36/15, e podem identificar: a) anticorpos, ou seja, uma resposta do organismo quando este teve contato com o vírus, recentemente (IgM) ou previamente (IgG); ou b) material genético (RNA) ou “partes” (antígenos) do vírus (RT-PCR). Existem os testes que usam sangue, soro ou plasma e os outros que precisam de amostras de secreções coletadas das vias respiratórias, como nasofaringe (nariz) e orofaringe (garganta)", "Esse termo vem sendo usado popularmente para os testes imunocromatográficos. No caso dos testes rápidos para o novo coronavírus, são dispositivos de uso profissional, manuais, de fácil execução, que não necessitam de outros equipamentos de apoio, como os que são usados em laboratórios, e que conseguem dar resu ltados entre 10 e 30 minutos.    Testes rápidos (IgM/IgG) podem auxiliar o mapeamento da população “imunizada” (que já teve o vírus ou foi exposta a ele), mas NÃO têm função de diagnóstico.", "RT-PCR (Reverse Transcription - Polymerase Chain Reaction) é um teste de Reação em Cadeia da Polimerase com Transcrição Reversa em tempo real que verifica a presença de material genético do vírus, confirmando que a pessoa se encontra com Cov id-19. Os testes de RT-PCR (padrão ouro) e de antígenos têm função diagnóstica, sendo o teste definitivo segundo a Organização Mundial da Saúde (OMS)1.", "Sim. Os testes rápidos registrados para a Covid-19 são de uso profissional e os seus resultados devem ser interpretados por um profissional de saúde legalmente habilitado e devidamente capacitado, conforme definido pelos conselhos profissionais da área da saúde e por políticas do Ministério da Saúde.  Esses testes NÃO devem ser feitos por leigos. Os testes em domicílio podem ser realizados, desde que executados por profissional legalmente habilitado vinculado a um laboratório clínico, posto de coleta ou serviço de saúde pública ambulatorial ou hospitalar.", "Como a produção de anticorpos aumenta a cada dia a partir do início da infecção pelo vírus, é preciso que haja uma quantidade mínima de anticorpos que o teste consiga detectar. Este período entre o início dos sintomas e a detecção dos anticorpos em exames é chamado de janela imunológica. Sendo assim, a imunocromatografia para anticorpos (IgM e IgG) é indicada para exames a partir de pelo menos oito dias após o início dos sintomas.  A utilização de testes rápidos antes desse período pode levar a resultados negativos mesmo nas pessoas que possuem o vírus e produziram anticorpos, sendo, portanto, um resultado “falso negativo”.", "Recomenda-se que tais testes sejam realizados em indivíduos que apresentem ou tenham tido os sintomas da Covid-19 há pelo menos oito dias.  Os testes RT-PCR devem ser utilizados quando houver sintomatologia compatível ou houver necessidade de confirmação da infecção. Os testes rápidos (IgM/IgG) têm relevante utilização no mapeamento do status imunológico de uma população (que já teve o vírus ou foi exposta a ele). Tal mapeamento pode contribuir de forma positiva no processo de relaxamento das medidas restritivas, ou seja, quando do controle pandêmico, o mapeamento imunológico terá significativa relevância por ocasião do retorno das atividades.", "Não. Testes rápidos (IgM/IgG) NÃO têm função de diagnóstico (confirmação ou descarte) de infecção por Covid-19.  O diagnóstico de Covid-19 deve ser feito por testes de RT-PCR.  • Testes rápidos positivos indicam que você teve contato recente com o vírus (IgM) ou que você já teve Covid-19 e está se recuperando ou já se recuperou (IgG), uma vez que indicam a presença de anticorpos (defesas do organismo). No entanto, os anticorpos só aparecem em quantidades detectáveis nos testes pelo menos oito dias depois da infecção. Ainda assim, o teste pode ser positivo indicando que você teve contato com OUTROS coronavírus e não com o Sars-CoV-2 / Covid-19 (falso positivo). Assim sendo, esse teste isolado não serve para diagnosticar (confirmar ou descartar) infecção por Covid-19. O diagnóstico da infecção pelo novo coronavírus deve ser feito por testes de RT-PCR.  • Os testes de RT-PCR (padrão ouro) e de antígenos têm função diagnóstica, sendo o RT-PCR o teste definitivo, segundo a Organização Mundial da Saúde (OMS). O profissional da saúde que estiver realizando o teste irá orientá-lo sobre o significado dos testes e os devidos encaminhamentos, conforme protocolos clínicos do Ministério da Saúde 2. É importante destacar que os pacientes, mesmo quando testados positivos para Covid -19, NÃO devem procurar hospitais ou ambulatórios, devendo permanecer em suas casas em qua rentena por 14 dias (isolamento) até a remissão da infecção, exceto se estiverem com sintomas graves, tal como dificuldade de respirar.", "Não. Testes rápidos (IgM/IgG) NÃO têm função de diagnóstico (confirmação ou descarte) de infecção por Covid-19. Diversos fatores influenciam os testes, tais como a sensibilidade/ especificidade e a condição do paciente (reposta imunológica). • Testes rápidos (IgM/IgG) negativos indicam que você não tem anticorpos contra a Covid-19. Considerando que esses anticorpos somente surgem em quantidade detectáveis alguns dias (pelo menos oito) depois da infecção, o teste somente tem alguma significância após esse período. Se sua carga imunológica (quantidade de anticorpos) for baixa, o teste pode ter um falso negativo. Assim sendo, esse teste isolado não serve para diagnosticar (confirmar ou descartar) infecção por Covid-19. O diagnóstico da doença deve ser feito por testes de RT-PCR. • Os testes de RT-PCR (padrão ouro) e de antígenos têm função diagnóstica, sendo o RT-PCR o teste definitivo, segundo a Organização Mundial da Saúde (OMS)3.", "Os ensaios imunocromatográficos são ensaios de simples execução que, geralmente, não requerem a utilização de equipamentos e permitem a visualização do resultado em poucos minutos (10 a 30 minutos em média, a depender de cada produto). Reforçamos que a informação apresentada nesses ensaios é quanto ao estado imunológico no momento da coleta da amostra. Há um período de janela imunológica, que é o intervalo de tempo entre a infecção e a produção de anticorpos em níveis detectáveis por um teste rápido, que precisa ser considerado e que não é inferior a oito dias após o início da infecção. Se a testagem ocorrer dentro do período de janela imunológica, o resultado do ensaio poderá ser negativo mesmo se a pessoa estiver contaminada (falso negativo). Por isso, é importante respeitar o intervalo entre os sintomas e a testagem, estar atento às informações das instruções de uso (que trazem orientações específicas sobre cada produto) e também que o teste seja executado e interpretado por profissional de saúde. Essa situação não corresponde necessariamente a uma falha no produto, mas à não observância da advertência quanto ao período adequado para a testagem.   Cabe mencionar, ainda, que cada produto possui características próprias quanto à execução, às precauções, ao limite de detecção e à interpretação dos resultados. Portanto, todas as informações dispostas nas instruções de uso devem ser seguidas e interpretadas por um profissional de saúde. Não há, até o momento, conhecimento científico sobre a duração dos anticorpos contra a Covid-19 no organismo e, portanto, não é possível assegurar proteção permanente para a infecção em pacientes com resultados de IgG positivo. As ações para redução do risco de transmissão do novo coronavírus dependem da participação de todos. Assim, independentemente dos resultados dos testes, a população deve estar atenta aos cuidados e às orientações divulgadas pelo Ministério da Saúde quanto ao uso de máscaras, o distanciamento mínimo entre as pessoas (evitar aglomerações) e o isolamento social na ocorrência de sintomas.", "s testes rápidos (IgM/IgG) têm relevante utilização no mapeamento do status imunológico de uma população (que já teve o vírus ou foi exposta a ele). Tal mapeamento pode contribuir de forma positiva no processo de relaxamento das medidas restritivas, ou seja, quando do controle pandêmico, o mapeamento imunológico terá significativa relevância por ocasião do retorno das atividades.", "Os testes são realizados a partir da coleta de uma amostra do paciente, podendo ser de secreções nasais, de garganta ou de sangue. O tempo necessário para a liberação do resultado depende do tipo de testagem utilizada. • Testes rápidos (IgM/IgG) geralmente utilizam sangue, soro ou plasma e demoram alguns minutos para liberar o resultado, a depender do produto (10 a 30 minutos).  • Os testes RT-PCR geralmente utilizam secreções respiratórias, coletadas por meio de swabs de orofaringe (garganta) ou nasofaringe (nariz ). São realizados em laboratórios clínicos e podem levar alguns dias para emissão de laudo. É importante destacar a importância de haver uma anamnese (avaliação e entrevista clínica) registrada, bem como a emissão de laudo técnico (sendo o resultado positivo ou negativo) para o paciente.", "O exame é feito com o uso de amostras de sangue, soro ou plasma. A metodologia utilizada é chamada de imunocromatografia, que é a geração de cor a partir de uma reação química entre antígeno (substância estranha ao organismo) e anticorpo (elemento de defesa do organismo). Os resultados obtidos são chamados de IgM e IgG.    Portanto, o IgM e o IgG são as defesas do organismo a um agente externo, como o vírus que provoca a Covid-19. Tendo em vista que o organismo demanda um tempo para a produção desses anticorpos (janela imunológica) a partir do contágio, a imunocromatografia é indicada para exames a partir de pelo menos oito dias após o início dos sintomas, a depender da limitação do teste e da indicação do fabricante. Esse tempo é necessário para assegurar que haverá anticorpos suficientes no organismo que possam ser detectáveis por esse método.    Os resultados dos testes rápidos devem ser interpretados por um profissional de saúde, considerando informações clínicas, sinais e sintomas do paciente, além de outros exames confirmatórios. Somente com esse conjunto de dados é possível fazer a avaliação e o diagnóstico ou descarte da doença.", "Sim. Uma das etapas do controle sanitário de produtos é o registro junto à Anvisa. Nesta fase, é exigida a apresentação de documentos da empresa, como Autorização de Funcionamento de Empresa (AFE) e Certificação de Boas Práticas de Fabricação (CBPF).   Também devem ser apresentados documentos sobre o produto a ser registrado, tais como ensaios clínicos, avaliação de desempenho, fluxo de produção, estudo de estabilidade e outros dados que indiquem a sua segurança, qualidade e eficácia. O objetivo é avaliar se o produto é capaz de dar o resultado para o qual foi desenvolvido.   É possível que haja produtos no mercado que não foram avaliados pela Anvisa.", "No dia 18 de março deste ano, a Anvisa publicou a Resolução da Diretoria Colegiada (RDC) 348/2020, que estabeleceu regras extraordinárias e temporárias para agilizar a avaliação de novos produtos, por meio da priorização da análise de pedidos de registro de testes para detecção do novo coronavírus.  A Anvisa tem direcionado toda sua energia e capacidade para avaliar a qualidade, a segurança e a eficácia dos testes para Covid-19. Considerando a emergência em saúde pública decorrente da pandemia de Covid-19, os procedimentos de regularização sanitária desses testes foram priorizados e flexibilizados, temporária e extraordinariamente, visando ampliar a oferta desses produtos e evitar o desabastecimento. Não adquira produtos pela internet. Você pode verificar a lista de testes aprovados pela Anvisa no portal da Agência.  Além dos testes registrados pela Anvisa, é possível encontrar produtos que tenham sido importados e adquiridos de forma excepcional, nos termos da Lei 13.979/2020, que autorizou de forma excepcional e temporária a importação de produtos suje itos à vigilância sanitária, desde que estes sejam registrados por autoridade sanitária estrangeira e estejam previstos em ato do Ministério da Saúde (VIII, art. 3º). De forma complementar, a Resolução da Diretoria Coleg iada (RDC) 356/2020, alterada pela RDC 379/2020, permitiu, de forma extraordinária e temporária,a aquisiçãode dispositivos médicos essenciais para o combate à Covid-19, novos e não regularizados pela Anvisa,desde que regularizados e comercializados em jurisdição membro do Fórum Internacional de Reguladores de Dispositivos Médicos (International Medical Device Regulators Forum –IMDRF), por órgãos e entidades públicas e privadas, bem como serviços de saúde,quando não disponíveis para o comércio dispositivos semelhantes regularizados na Agência (art. 9º), sendo que os responsáveis pelas importações de kits para diagnóstico devem enviar, em um prazo máximo de 5 (cinco) dias contados da data do desembaraço da carga, uma amostra de, no mínimo, 100 unidades de cada lote importado para análise do Instituto Nacional de Controle de Qualidade em Saúde da Fundação Oswaldo Cruz (INCQS/Fiocruz).", "O controle do risco sanitário envolve ações pré e pós-mercado, ou seja, que acontecem antes do produto ser comercializado e que decorrem do uso e monitoramento do comportamento do produto após a sua comercialização.  O registro é parte da atuação do controle sanitário que ocorre antes que o produto sej a comercializado no mercado nacional. A partir dele pode-se verificar se as empresas envolvidas nos processos fabris e nas atividades de importação encontram-se habilitadas para a realização dessas atividades.   Os produtos para diagnóstico in vitro da Covid-19 são de uso profissional e classificados como de alto risco aos indivíduos e à saúde pública (classe III), e as empresas que os fabricam devem atender aos critérios de Boas Práticas de Fabricação estabelecidos pela RDC 16/2013. A Anvisa certifica as unidades fabris que cumprem os requisitos indicados nessa resolução , relacionados ao sistema de qualidade das empresas e aos controles aplicados no desenvolvimento e na fabricação dos produtos.  Além disso, o registro também engloba a avaliação de desempenho dos produtos. Para tanto, são apresentadas informações, na forma de um dossiê técnico documental, que permitem a avaliação da confiabilidade dos resultados e da efetividade diagnóstica do produto. Os fabricantes precisam demonstrar como foram realizados os testes de desempenho e a qualificação das amostras utilizadas, além de evidências clínicas, tendo em vista o tipo de produto, a indicação de uso e a metodologia. Durante o processo de registro de cada produto são verificadas as rotulagens e as instruções de uso, para que contemplem as informações essenciais para a sua utilização segura e orientem o profissional de saúde quanto aos procedimentos para a execução do ensaio, o tipo de amostra, a interpretação dos resultados, as limitações e as advertências, entre outras informações relevantes. No Brasil, o registro de produto importado deve ser feito por uma empresa nacional que tenha autorização do fabricante para representá-lo no país. Essa documentação também é parte do processo de avaliação.  A concessão do registro, portanto, é a primeira etapa do controle sanitário. É responsabilidade dos fabricantes e importadores disponibilizarem no mercado produtos que estejam em estrita conformidade com as informações aprovadas no registro. A Anvisa e as Vigilâncias Sanitárias estaduais e municipais continuam monitorando o comportamento dos produtos após a sua comercialização, seja por meio de queixas técnicas ou pela avaliação laboratorial de desempenho.", "São testes desenvolvidos por laboratórios clínicos, conforme seus próprios protocolos. O desenvolvimento deste tipo de teste segue normas da Anvisa (Resolução da Diretoria Colegiada – RDC 302/2005) e a produção é exclusivamente para uso próprio do estabelecimento, em pesquisa ou apoio diagnóstico. Por isso, não podem ser comercializados.", "Sim.  A Anvisa lançou a plataforma Business Intelligence (BI) da fila completa de produtos de diagnóstico in vitro para Covid-19. A plataforma de inteligência empresarial proporciona transparência à situação dos pedidos protocolados na Agência.   A novidade promove a consulta a diversos dados, como por exemplo a quantidade de pedidos deferidos, indeferidos, em análise, aguardando o Certificado de Boas Práticas de Fabricação (CBPF), entre outros. Acesse o BI de produtos de diagnóstico in vitro para Covid-19. Além disso, a Anvisa disponibilizou um informe contendo a relação de produtos para diagnóstico in vitro para Covid-19 regularizados. Essa relação apresenta informações técnicas gerais como metodologia, tipos de amostra e alvos dos produtos de forma bastante objetiva, bem como hiperlinks para a página de consulta de produtos regularizados do portal da Agência, onde poderão ser acessadas as instruções de uso desses produtos. O informe é atualizado sempre que novos produtos são regularizados e está disponível no portal da Anvisa.  A Anvisa esclarece que a produção e a oferta dos testes dependem da capacidade de cada empresa que recebe o registro.", "Há testes sorológicos, que utilizam diferentes metodolog ias para detectar anticorpos e fazem o uso de amostras como sangue, soro ou plasma, e testes moleculares, que detectam a presença do antígeno, como o RT-PCR.   A metodologia chamada imunocromatografia (geração de cor a partir de uma reação entre o antígeno e o anticorpo) vem sendo utilizada para o coronavírus para detectar anticorpos produzidos pelo próprio organismo do paciente em resposta à infecção pela Covid-19, chamados de IgM e IgG. Esses indicadores revelam se a pessoa teve contato com o vírus.   Tendo em vista que o organismo demanda um tempo para a produção desses anticorpos (janela imunológica) a partir do contágio, a imunocromatografia é indicada para exames a partir de pelo menos oito dias após o início dos sintomas.  Também foram aprovados testes para detecção de anticorpos com o uso de outras metodologias que requerem a utilização de equipamentos disponíveis em laboratórios, como, por exemplo, ELISA, que se baseia numa reação enzimát ica; imunoensaio quimioluminescente –  CLIA, que torna a reação antígeno-anticorpo visível por uma reação química; e imunofluorescência, no qual a leitura do resultado é feita a partir da fluorescência formada na reação do antígeno com o anticorpo, entre outras.  Entre os testes aprovados também há os que usam o método RT-PCR (sigla em inglês para transcrição reversa seguida de reação em cadeia da polimerase), recomendado para o diagnóstico da doença. Esse tipo de teste se baseia na detecção de fragmentos do material genético do vírus e revela se a pessoa está doente no momento da realização do exame, porém não detecta contágios passados.  Saiba quais são os produtos para diagnóstico in vitro de Covid-19 regularizados.", "Sim. No dia 18 de março deste ano, a Agência publicou a Resolução da Diretoria Colegiada (RDC) 348/2020, que estabeleceu regras extraordinárias e temporárias para agilizar a avaliação de novos produtos, por meio da priorização da análise de pedidos de registro de testes para detecção do novo coronavírus (Sars-CoV-2).  A ideia não é avaliar e aprovar produtos de forma automática, pois o rigor sanitário sempre deve existir, mas sim dar mais rapidez ao processo. A medida faz parte das ações estratégicas para viabilizar produtos que possam ser utilizados no enfrentamento da pandemia de Covid-19.", "É importante esclarecer que, para a concessão desses registros, não são realizados ensaios laboratoriais prévios. No entanto, trata-se de produtos de uso profissional e os laboratórios que os utilizam tem como prática regulamentada pela Anvisa a aplicação de controles internos para avaliar se o sistema analítico está operando dentro dos limites de tolerância predefinidos, de forma a minimizar os riscos envolvidos na utilização de produtos diagnósticos.  No enfrentamento da emergência em saúde pública provocada pela Covid-19, o Instituto Nacional de Controle de Qualidade em Saúde (INCQS/Fiocruz) e o Ministério da Saúde têm realizado a avaliação dos produtos adquiridos para definir a estratégia de uso e distribuição.  Também é importante ressaltar que a responsabilidade pelas informações prestadas à Anvisa é do detentor do registro do produto e o controle do desempenho do processo fabril é previsto como preceito das Boas Práticas de Fabricação (BPFs).  O teste rápido de anticorpos para o novo coronavírus (Sars-CoV-2) pode ser usado como apoio para a avaliação do estado imunológico de pacientes que apresentem sintomas da Covid-19. Basicamente, esse tipo de exame aponta se a pessoa teve ou não contato com o vírus. Quando uma pessoa entra em contato com o vírus, o organismo inicia a produção de anticorpos como um mecanismo de defesa. No entanto, é preciso aguardar alguns dias até que a quantidade desses anticorpos seja detectável em um teste (janela imunológica) e estar atento às orientações das instruções de uso dos testes.", "Por qual instituição foram avaliados? A Anvisa, em parceria com o INCQS/Fiocruz, estabeleceu um programa de monitoramento dos testes rápidos. Esse programa não substitui as ações regulares de controle sanitário e não é condição para uso de produtos registrados na Agência, mas permite acompanhar o comportamento dos produtos frente às informações declaradas nas instruções de uso. Qualquer desempenho não esperado dos produtos identificado pelos serviços de saúde deve ser notificado à Anvisa por meio do Sistema de Notificações em Vigilância Sanitária (Notivisa) e é parte do controle sanitário pós-mercado. Além disso, a Agência atua com as autoridades reguladoras internacionais para discutir temas como análises laboratoriais de produtos para diagnóstico e compartilhamento dos resultados.", "Ou tem algum sendo utilizado pela rede privada? Quem deve VALIDAR os testes rápidos usados pela rede privada?  Não há determinação expressa, em regulamentos técnicos ou em leis, que estabeleça que os lotes de kits para a Covid-19 registrados na Anvisa tenham que ser previamente validados. Portanto, a ausência de laudo de validação não impede o uso desses produtos. Os serviços de saúde são regulados pela Anvisa e o acompanhamento do desempenho dos produtos é atividade inerente ao gerenciamento das tecnologias em saúde utilizadas nos serviços, que incluem os produtos para a saúde, de modo a garantir a sua rastreabilidade, qualidade, eficácia, efetividade e segurança e, no que couber, desempenho, desde a entrada no estabelecimento de saúde até seu destino final, incluindo o planejamento dos recursos físicos, materiais e humanos, bem como da capacitação dos profissionais envolvidos no seu processo (RDC 02/2010).", "Não há determinação expressa, em regulamentos técnicos ou em leis, que estabeleça que os testes rápidos para a Covid-19 registrados na Anvisa tenham que ser previamente validados antes de serem utilizados. Portanto, a ausência de validação não impede o uso desses produtos. Destaca-se que é responsabilidade dos fabricantes e importadores disponibilizarem no mercado produtos que estejam em estrita conformidade com as informações aprovadas no registro. Quando observados desvios de qualidade nos produtos, os serviços/profissionais de saúde devem notificar a Anvisa para a adoção de medidas cabíveis. No entanto, trata -se de produtos de uso profissional e os estabelecimentos de saúde que os utilizam têm como prática regulamentada pela Anvisa a aplicação de controles internos para avaliar se o sistema está operando dentro dos limites de tolerância predefinidos, de forma a minimizar os riscos envolvidos na utilização de produtos.", "A RDC 302/2005, que dispõe sobre o Regulamento Técnico para Funcionamento de Laboratórios Clínicos, determina, no item 6.2.13 de seu anexo, que a execução dos testes laboratoriais remotos – TLR (point-of-care) e de testes rápidos pode ser realizada próximo ao paciente e deve 'estar vinculada a um laboratório clínico, posto de coleta ou serviço de saúde pública ambulatorial ou hospitalar.   Assim, os testes rápidos (IgM/IgG) para a Covid-19 podem ser feitos em estabelecimentos de assistência à saúde com licenciamento para esta atividade e por um profissional de saúde devidamente habilitado e treinado para esta finalidade. Os testes geralmente são feitos em laboratórios clínicos, podendo haver coleta no local onde estiver o paciente, incluindo sua casa. É importante destacar que o teste deve ser realizado em espaço priva tivo, seguindo todos os protocolos clínicos de gestão do paciente e manipulação de amostras, em especial aqueles relacionados à proteção sanitária (uso de equipamentos de proteção individual – EPIs).  Para maiores informações sobre a utilização de testes rápidos em farmácias, consulte as informações constantes no portal da Anvisa.", "Até o momento, os testes para Covid-19 estão registrados na Anvisa para uso profissional, devendo ser fornecidos apenas por meio de distribuidoras de produtos para saúde legalmente autorizadas para tal. Dessa forma, as empresas que podem fornecer es ses testes são aquelas que atuam no ramo de atividade que contemple o “comércio atacadista de produtos para saúde”, devendo possuir Autorização de Funcionamento de Empresa e licença sanitária que contemplem a atividade de distribuição de produtos para saúde. Conforme a RDC 16/2014, a Autorização de Funcionamento de Empresa para produtos para saúde é exigida para o distribuidor ou comércio atacadista, compreendendo o comércio de produtos para saúde, em quaisquer quantidades, realizadas entre pessoas jurídicas ou a profissionais para o exercício de suas atividades.", "Podem ser comercializados testes para pessoas físicas? As distribuidoras de produtos para saúde podem comercializar os testes para Covid-19 a pessoas jurídicas ou profissionais, para o exercício de suas atividades em serviços de saúde destinados à execução dessa atividade. Assim, por se tratar de produtos para uso profissional, não é permitido o seu comércio à população em geral. Os testes podem ser comercializados para pessoas jurídicas cujas atividades são destinadas à prestação de serviços de saúde à população, como laboratórios, hospitais e outros estabelecimentos de saúde, públicos ou privados.", "Até o momento, não há restrições da Anvisa em relação ao comércio de testes para Covid-19 para serviços de saúde públicos ou privados. Orientações adicionais podem ser encontradas em relação às políticas de distribuição dos testes por meio do Ministério da Saúde. A RDC 302/2005 define o uso de testes laboratoriais remotos (TLR) somente sob responsabilidade do laboratório clínico ou das unidades públicas de saúde. Dessa forma, fica proibida a venda (por se assumir que a compra é para uso) para outro tipo de serviço de saúde.", "As distribuidoras de produtos para saúde apenas podem comercializar os testes para Covid-19 a pessoas jurídicas ou profissionais para o exercício de suas atividades.  Assim, por se tratar de produtos para uso profissional, o seu comércio apenas é permitido a pessoas jurídicas cujas atividades são destinadas à prestação de serviços de saúde à população, como laboratórios, hospitais e outros estabelecimentos de saúde, públicos ou privados.   Dessa forma, às distribuidoras de produtos para saúde não é permitido comercializar testes para Covid-19 a empresas que não se enquadrem nesta hipótese.", "Farmácias e drogarias não podem comercializar/vender testes para Covid-19, pois não são distribuidoras de produtos para saúde e os testes não se destinam a uso leigo. Entretanto, a farmácia ou drogaria devidamente licenciada para a prestação de serviços de assistência à saúde pode utilizar os testes rápidos para Covid-19, mediante responsabilidade técnica e condições sanitárias (de estrutura, processos e pessoas) adequadas, conforme determinado na RDC 377/2020 e nas orientações estabe lecidas na Nota Técnica 96/2020 e na Nota Técnica 97/2020. A disponibilidade dos testes rápidos em farmácias e drogarias dependerá da decisão do estabelecimento de oferecer ou não este serviço de assistência à saúde. A norma da Anvisa não estabelece obrigatoriedade do serviço em questão nessas unidades. Outro fator para a disponibilização é a produção e a entrega dos testes rápidos pela empresa que obteve o registro junto à Agência.", "Os testes rápidos, apesar de serem de uso relativamente simples, são de uso profissional e sua execução requer o cumprimento de protocolos e diretrizes técnicas de controle, rastreabilidade e registros. A RDC 302/2005, que dispõe sobre o Regulamento Técnico para Funcionamento de Laboratórios Clínicos, determina, no item 6.2.13 de seu anexo, que a execução dos testes laboratoriais remotos – TLR (point-of-care) e de testes rápidos (IgM/IgG) pode ser realizada próximo ao paciente (nas instalações das empresas, por exemplo) e deve estar vinculada a um laboratório clínico, posto de coleta ou serviço de saúde pública ambulatorial ou hospitalar.", "Sim. É recomendada a testagem em massa com testes rápidos (IgM/IgG) em grupos envolvidos em atividades essenciais, tais como profissionais que atuam nas áreas de saúde (hospitais, farmácias, vigilância sanitária etc.), segurança pública, limpeza urbana, de suprimento (postos de combustíveis, supermercados etc.), dentre outras. Nesses casos, é necessário que os testes sejam feitos por profissionais de saúde devidamente habilitados e treinados e que estes estejam vinculados a um laboratório clínico, posto de coleta ou serviço de saúde pública ambulatorial ou hospitalar. Em todos os casos devem ser observadas as instruções de uso do kit diagnóstico e as LIMITAÇÕES dos testes devem ser devidamente consideradas. Testes RT-PCR somente devem ser realizados sob indicação médica.", "Sim. A testagem em massa com testes rápidos (IgM/IgG) em grupos vulneráveis , como penitenciárias, institutos de longa permanência – ILPs (asilos) etc. somente pode ocorrer sob supervisão e/ou autorização das autoridades de saúde. Deve-se ponderar cuidadosamente a relação de benefícios versus riscos de tal ação.   Em todos os casos, as LIMITAÇÕES dos testes devem ser devidamente consideradas. Testes RT-PCR somente devem ser realizados sob indicação médica.", "Quaisquer indícios de irregularidade/falsificação devem ser imediatamente comunicados à Anvisa (Ouvidoria) ou à autoridade sanitária local. É importante destacar que a Anvisa estabeleceu um amplo e robusto programa de fiscalização e verificação dos produtos disponíveis no mercado, visando a avaliação da conformidade dos produtos aos requisitos mínimos de qualidade, segurança e eficácia.", "Para notificação de produtos para diagnóstico de Cov id-19, temos os seguintes links para notificação pelos profissionais:  •         Notiﬁcação de queixa técnica de produtos para diagnóstico de Covid-19 que têm número de registro, por profissionais de saúde: https://www8.anvisa.gov.br/notivisa/frmlogin.asp  •         Notiﬁcação de evento adverso de produtos para diagnóstico de Covid-19 que têm número de registro, por pacientes, familiares, acompanhantes e cuidadores: http://www16.anvisa.gov.br/notivisaServicos/cidadao/notificacao/evento-adverso  •         Os casos suspeitos ou conﬁrmados de evento adverso e de queixa técnica associados aos produtos para diagnóstico de Covid-19  fabricados, importados e adquiridos por força da RDC 356/2020 devem ser notiﬁcados por meio do formulário eletrônico “Notiﬁcação de Evento Adverso e Queixa Técnica (Covid-19) – RDC 356/2020 e RDC 379/2020”, disponível no endereço eletrônico https://pesquisa.anvisa.gov.br/index.php/114757?lang=pt-BR   •         As  denúncias em relação a práticas comerciais fraudulentas, bem como importação ou distribuição irregular, devem ser encaminhadas à Anvisa por meio do Sistema da Ouvidoria. Nesse caso, o notificante pode optar por não se identificar, mantendo o seu anonimato."]

In [88]:
respostas

['Os testes para Covid-19 são produtos para diagnóstico de uso in vitro, nos termos da RDC 36/15, e podem identificar: a) anticorpos, ou seja, uma resposta do organismo quando este teve contato com o vírus, recentemente (IgM) ou previamente (IgG); ou b) material genético (RNA) ou “partes” (antígenos) do vírus (RT-PCR). Existem os testes que usam sangue, soro ou plasma e os outros que precisam de amostras de secreções coletadas das vias respiratórias, como nasofaringe (nariz) e orofaringe (garganta)',
 'Esse termo vem sendo usado popularmente para os testes imunocromatográficos. No caso dos testes rápidos para o novo coronavírus, são dispositivos de uso profissional, manuais, de fácil execução, que não necessitam de outros equipamentos de apoio, como os que são usados em laboratórios, e que conseguem dar resu ltados entre 10 e 30 minutos.    Testes rápidos (IgM/IgG) podem auxiliar o mapeamento da população “imunizada” (que já teve o vírus ou foi exposta a ele), mas NÃO têm função de dia

In [86]:
len(perguntas) == len(respostas)

True

In [91]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./ceara-gov.csv', index = False)

# Código 6

In [108]:
html_soup = RequestSoup("https://saojosedobelmonte.pe.gov.br/perguntas-e-respostas-tire-suas-duvidas-sobre-o-novo-coronavirus/")

In [109]:
soup = html_soup.find_all('div', class_='container-fluid m-100')[0].find_all(['p','h2'])[:-5]
soup

[<p class="categoria-single">Saúde</p>,
 <p class="resumo-single">Secretaria da Saúde de São José do Belmonte orienta como se proteger e o que fazer em casos de suspeita da doença O Governo de São José do Belmonte continua efetuando diversas ações contra a propagação do novo coronavírus. O...</p>,
 <p><em><strong>Secretaria da Saúde de São José do Belmonte orienta como se proteger e o que fazer em casos de suspeita da doença</strong></em></p>,
 <p>O Governo de São José do Belmonte continua efetuando diversas
 ações contra a propagação do novo coronavírus. O novo coronavírus é denominado
 oficialmente como COVID-19, sigla em inglês para “<em>coronavirus disease 2019</em>” (doença por coronavírus
 2019, em tradução livre). </p>,
 <p>Saiba mais sobre o coronavírus, veja como se proteger e quais
 procedimentos adotar diante de casos suspeitos.</p>,
 <h2><strong>O que é o coronavírus?</strong></h2>,
 <p>Os coronavírus são uma grande família viral, conhecidos desde
 meados de 1960, que causa

In [114]:
perguntas = []
posi = []
for i in range(len(soup)):
    if("?" in soup[i].text):
        perguntas.append(soup[i].text.replace("\n",""))
        posi.append(i)

In [115]:
perguntas

['O que é o coronavírus?',
 'Qual é a diferença entre o novo coronavíruspara os outros vírus?',
 'Existe vacina para prevenção ao coronavírus?',
 'Quais os sintomas do coronavírus?',
 'Qual (is) remédio(s) a pessoa com COVID-19 devetomar?',
 'O que é o “período de incubação”?',
 'Como ocorre a transmissão do coronavírus?',
 'O coronavírus pode matar?',
 'Como se prevenir contra o COVID-19?',
 'Como é a prevenção contra o coronavírus paraos profissionais de saúde?',
 'O que fazer em caso de sintomas?',
 'Qual o tratamento indicado para uma pessoa comCOVID-19?',
 'Como é feito o diagnóstico do COVID-19?',
 'Qual é a definição de pessoasmonitoradas?',
 'Por que tanto casos em monitoramento?',
 'Qual é a definição de caso notificado?',
 'Qual é a definição de caso suspeito?',
 'Quanto tempo dura a recuperação de uma pessoacom COVID-19?',
 'Qual é a definição de caso confirmado?',
 'O que é o “comunicante” de um caso confirmado?',
 'Existem casos de pessoas infectadas que nãoestão no boleti

In [135]:
intervalos = []
for i in range(len(posi)):
    if(i != len(posi)-1):
        intervalos.append((posi[i]+1,posi[i+1]))
    else:
        intervalos.append((posi[i]+1,len(soup)))

In [137]:
respostas = []
for i in intervalos:
    text = ""
    for j in range(i[0],i[1]):
        text += soup[j].text.replace("\n","").replace("\xa0","")
    respostas.append(text)

In [138]:
perguntas

['O que é o coronavírus?',
 'Qual é a diferença entre o novo coronavíruspara os outros vírus?',
 'Existe vacina para prevenção ao coronavírus?',
 'Quais os sintomas do coronavírus?',
 'Qual (is) remédio(s) a pessoa com COVID-19 devetomar?',
 'O que é o “período de incubação”?',
 'Como ocorre a transmissão do coronavírus?',
 'O coronavírus pode matar?',
 'Como se prevenir contra o COVID-19?',
 'Como é a prevenção contra o coronavírus paraos profissionais de saúde?',
 'O que fazer em caso de sintomas?',
 'Qual o tratamento indicado para uma pessoa comCOVID-19?',
 'Como é feito o diagnóstico do COVID-19?',
 'Qual é a definição de pessoasmonitoradas?',
 'Por que tanto casos em monitoramento?',
 'Qual é a definição de caso notificado?',
 'Qual é a definição de caso suspeito?',
 'Quanto tempo dura a recuperação de uma pessoacom COVID-19?',
 'Qual é a definição de caso confirmado?',
 'O que é o “comunicante” de um caso confirmado?',
 'Existem casos de pessoas infectadas que nãoestão no boleti

In [139]:
respostas

['Os coronavírus são uma grande família viral, conhecidos desdemeados de 1960, que causam infecções respiratórias em seres humanos e emanimais.Geralmente, infecções por coronavírus causam doençasrespiratórias leves a moderadas, semelhantes a um resfriado comum. Porém,alguns coronavírus podem causar doenças graves com impacto em termos de saúdepública, como já verificado com a Síndrome Respiratória Aguda Grave (SARS),identificada em 2002, e a Síndrome Respiratória do Oriente Médio (MERS),identificada em 2012.',
 'A doença provocada pelo novo coronavírus chama-se COVID-19,sigla em inglês para “coronavirusdisease 2019” (doença por coronavírus 2019, em tradução livre).Os primeiros casos foram registrados inicialmente na China, nofinal de 2019. Há registros em outros locais do mundo, com casos de mortes.',
 'Até o momento, não. No entanto, cientistas ao redor do mundo jáiniciaram pesquisas para desenvolvimento de vacina. Ainda é precoce indicar see quando ela estará disponível.',
 'Os sinai

In [142]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./PE-belmonte.csv', index = False)

# Código 7

In [33]:
html_soup = requestSoup("http://rj.corens.portalcofen.gov.br/covid-19-guia-de-perguntas-e-respostas-frequentes_18302.html")

In [34]:
soup = html_soup.find_all('article', class_ = 'post-18302 post type-post status-publish format-standard hentry category-noticias tag-enfermagemcontraacovid19 tag-juntoscontraocoronavirus')[0]

In [35]:
soup = soup.find_all('div', class_ = 'content')[0].find_all('span')[:-9]

In [36]:
perguntas = []
posi = []
for i in range(len(soup)):
    if("?" in soup[i].text):
        perguntas.append(soup[i].text.replace("\n","").replace("* ",""))
        posi.append(i)

In [37]:
perguntas

['Quais são os grupos de risco relacionados à Covid-19?',
 'Faço parte do grupo de risco para Covid-19. O que devo fazer?',
 'Devo ser afastada do trabalho se estou gestante?',
 'O profissional de enfermagem com sintomas, que cumpriu o isolamento domiciliar após 14 dias pode retornar ao trabalho?',
 'O que fazer com os profissionais de enfermagem que tiveram contato com algum enfermeiro, técnico ou auxiliar que foi diagnosticado com a Covid-19?',
 'O profissional de enfermagem escalado para assistência a paciente suspeito ou confirmado deve ser exclusivo?',
 'É possível a utilização de máscaras caseiras nos ambientes de trabalho?',
 'Em que condições deve ser utilizada a máscara caseira?']

In [38]:
intervalos = getIntervalos(posi,soup)

In [39]:
respostas = []
for i in intervalos:
    text = ""
    for j in range(i[0],i[1]):
        text += soup[j].text.replace("\n","").replace("\xa0","")
    respostas.append(text)

In [40]:
respostas

['O grupo de risco é composto por: idosos acima de 60 anos, pessoas com Doença Pulmonar Obstrutiva Crônica (DPOC), asma, pneumopatia, doenças cerebrovascular, cardiopatia (incluindo hipertensão arterial, diabéticos), pessoas com insuficiência renal, imunodeprimidos, gestantes, nutrizes e puérperas.',
 'Pedir ao seu médico um laudo que comprove sua condição e encaminhar o laudo para o enfermeiro responsável pelo serviço. O Responsável Técnico ou coordenador de Enfermagem deverá encaminhar para o serviço de saúde do trabalhador (se for o caso) e, de acordo com a decisão do gestor, avaliar acerca do deslocamento de setor para uma área não-Covid, para trabalhar de casa (se possível) ou afastamento completo das atividades.',
 'De acordo com a Lei nº 13.467/2017, artigo 394-A, inciso I, a gestante deverá ser afastada de: atividades consideradas insalubres em grau máximo, enquanto durar a gestação; atividades consideradas insalubres em grau médio ou mínimo, quando apresentar atestado de saúde

In [41]:
comparacao(perguntas,respostas)

True

In [43]:
pd.DataFrame({"Perguntas" : perguntas, "Respostas" : respostas}).to_csv('./RJ-corens.csv', index = False)

# Código 8

In [70]:
with open('./img/3.pdf', 'rb') as f:
        extracted_text = slate.PDF(f)
        extracted_text = extracted_text[0]

In [71]:
soup = extracted_text.split("\n\n")
soup

['conselho profissional, serão de cobertura obrigatória, uma vez atendida a diretriz de utilização do ',
 'procedimento, quando houver, e de acordo com as regras pactuadas no contrato estabelecido entre ',
 'a operadora e o prestador de serviços. Do mesmo modo, caso o plano do beneficiário tenha previsão ',
 'de livre escolha de profissionais, mediante reembolso, o atendimento realizado por meio de tal ',
 'modalidade também terá cobertura e deverá ser reembolsado, na forma prevista no contrato.   ',
 '15 - O que o beneficiário deve fazer se a instituição de saúde se recusar a atender à distância?  ',
 'O beneficiário deve sempre entrar em contato com a operadora do seu plano de saúde para que ',
 'verificar se o estabelecimento e/ou profissional no qual pretende ter atendimento está credenciado ',
 'para realização de procedimentos e serviços à distância. Caso o plano do beneficiário tenha ',
 'previsão de livre escolha de profissionais, mediante reembolso, o atendimento realizado à d

In [73]:
soup = ['9 - Caso tenha dúvidas, como o beneficiário de plano de saúde deve ser informar? ',
 'O usuário deve sempre procurar informações e orientações junto à operadora do seu plano de ',
 'saúde. A ANS está orientando as empresas para que disponibilizem em seus portais na internet e ',
 'disseminem através de seus canais de relacionamento informações sobre o atendimento e a ',
 'realização do exame e ofereçam canais de atendimento específicos para prestar esclarecimentos e ',
 'informações sobre a doença aos seus usuários. A ANS também presta informações e registra as ',
 'reclamações dos beneficiários através dos seguintes canais de atendimento: Disque ANS - 0800 701 ',
 '9656; Central de Atendimento para deficientes auditivos - 0800 021 2105; Fale Conosco no portal ',
 'da Agência – www.ans.gov.br.  ',
 'TELESSAÚDE ',
 '10 - A telemedicina pode ser utilizada pelos beneficiários de planos de saúde?  ',
 'Sim, nesse período de pandemia, seguindo orientação das autoridades de saúde para priorizar o ',
 'isolamento social e evitar a sobrecarga dos serviços de saúde, os beneficiários de planos de saúde ',
 'devem, sempre que possível, procurar aconselhamento médico ou de outros profissionais de saúde ',
 'por telefone ou outras tecnologias que possibilitem, de forma não presencial, a troca de ',
 'informações para diagnóstico, tratamento e prevenção de doenças. ',
 '11 - Como o usuário deve proceder para ter esse tipo de atendimento?  ',
 'Para ser atendido remotamente, o beneficiário deve entrar em contato com a operadora do seu ',
 'plano de saúde, que lhe informará quais prestadores de serviços estão credenciados para realizar ',
 'esse tipo de atendimento. É importante que os atendimentos realizados por intermédio ',
 'da telessaúde devem sempre observar os limites autorizados pelas normativas da ANS, dos ',
 'respectivos conselhos profissionais e Ministério da Saúde, e pela recente lei federal nº ',
 '13.989/2020.  ',
 '12 - Quais canais de comunicação podem ser usados para o atendimento à distância?  ',
 'Os canais devem ser previamente pactuados entres as operadoras e seus respectivos prestadores ',
 'de serviços de saúde, utilizando recursos de tecnologia da informação e comunicação na forma ',
 'prevista nas resoluções dos respectivos conselhos de profissionais de saúde, Ministério da Saúde e ',
 'na lei federal nº 13.989/2020. Esses canais e instrumentos devem garantir a segurança e a ',
 'privacidade dos dados de saúde dos beneficiários, que são informações sensíveis protegidas por ',
 'legislação especial.  ',
 '13 - Os hospitais e clínicas são obrigados a oferecer a opção de telemedicina?  ',
 'Os prestadores de serviços de saúde não são obrigados a realizar este tipo de atendimento. Por ',
 'esta razão, é necessário que haja prévia pactuação entre a operadora de plano de saúde e cada um ',
 'dos prestadores que pretendem realizar atendimentos via telessaúde. Enquanto perdurar a ',
 'situação de pandemia, essa prévia pactuação poderá ser feita através de qualquer instrumento, ',
 'como e-mails ou troca de mensagens eletrônicas no site da operadora, desde que discriminem: os ',
 'serviços prestados, os valores e ritos de pagamento a serem observados para os serviços de ',
 'telessaúde.  ',
 '14 - A operadora pode cobrar um valor adicional pelo atendimento a distância?  ',
 'Não. Os atendimentos realizados pelos profissionais de saúde que compõem a rede assistencial do ',
 'plano, aos seus beneficiários, por meio de comunicação à distância, na forma autorizada por seu ',
 'conselho profissional, serão de cobertura obrigatória, uma vez atendida a diretriz de utilização do ',
 'procedimento, quando houver, e de acordo com as regras pactuadas no contrato estabelecido entre ',
 'a operadora e o prestador de serviços. Do mesmo modo, caso o plano do beneficiário tenha previsão ',
 'de livre escolha de profissionais, mediante reembolso, o atendimento realizado por meio de tal ',
 'modalidade também terá cobertura e deverá ser reembolsado, na forma prevista no contrato.   ']

In [74]:
conjuntoInformacoes = getPerguntasPDF(soup)

In [75]:
interval = getIntervalos(conjuntoInformacoes[1],soup)

In [76]:
interval

[(1, 10), (11, 16), (17, 23), (24, 30), (31, 38), (39, 46)]

In [51]:
#respostas = getRespostasPDF(interval,soup)

In [54]:
#perguntas = conjuntoInformacoes[0]

In [77]:
perguntas.extend(conjuntoInformacoes[0])

In [78]:
perguntas

['1 - Caso suspeite que esteja infectado com o Coronavírus, como o beneficiário de plano de saúde deve proceder? ',
 '2 - O exame para detecção do Coronavírus é coberto pelos planos de saúde? ',
 '3 - Em que casos deve ser feito o exame para detecção do vírus (RT -PCR)? ',
 '4 – Há outros exames na lista de coberturas que auxiliam no diagnóstico e tratamento da Covid-19?',
 '5 - Os planos de saúde cobrem o tratamento dos problemas de saúde causados pelo coronavírus? ',
 '6 - Os planos de saúde podem exigir o cumprimento de carência nos casos relacionados à Covid -19? ',
 '7 - Procedimentos não urgentes devem ser adiados durante a pandemia? E internações e cirurgias eletivas devem ser suspensas?  ',
 '8 - Os planos de saúde podem suspender o atendimento por falta de pagamento? ',
 '9 - Caso tenha dúvidas, como o beneficiário de plano de saúde deve ser informar? ',
 '10 - A telemedicina pode ser utilizada pelos beneficiários de planos de saúde?  ',
 '11 - Como o usuário deve proceder par

In [85]:
perguntas.extend(['15 -O que o beneficiário deve fazer se a instituição de saúde se recusar a atender à distância?','16 -Como será feito o procedimento de autorização de consulta junto ao plano de saúde, uma vez que o paciente não poderá assinar a guia?'])

In [79]:
respostas.extend(getRespostasPDF(interval,soup))

In [83]:
respostas

['Cada operadora de plano de saúde definiu o melhor fluxo para atendimento de seus beneficiários, portanto, a orientação é que o usuário que desconfie que está com Coronavírus entre primeiramente em contato com a operadora e se informe sobre os locais e orientações para atendimento. ',
 'Sim, o exame SARS-CoV-2 (CORONAVÍRUS COVID-19) – pesquisa por RT – PCR (com diretriz de utilização) foi incluído pela ANS no Rol de Procedimentos e Eventos em Saúde e é, portanto, de cobertura obrigatória aos beneficiários de planos de saúde com segmentação ambulatorial, hospitalar ou referência. A medida vale desde o dia 13/03/2020. ',
 'O exame deve ser feito nos casos em que a condição de saúde beneficiário se enquadrar como caso suspeito ou provável de Covid-19, conforme indicação médica. O médico assistente deverá avaliar o paciente de acordo com o protocolo e as diretrizes definidas pelo Ministério da Saúde, a quem compete definir os casos enquadrados como suspeitos ou prováveis de doença pelo Co

In [86]:
respostas.extend(['O beneficiário deve sempre entrar em contato com a operadora do seu plano de saúde para que verificar se o estabelecimento e/ou profissional no qual pretende ter atendimento está credenciado para realização de procedimentos e serviços à distância. Caso o plano do beneficiário tenha previsão de livre escolha de profissionais, mediante reembolso, o atendimento realizado à distância deverá ser reembolsado na forma prevista no contrato.','Essa questão deve ser ajustada previamenteentre as operadoras e os prestadores de serviços integrantes de sua rede, destacando que a ANS admite a possibilidade da verificação da elegibilidade e solicitação da autorização sem a assinatura do paciente. '])

In [88]:
criarDataframe(perguntas,respostas).to_csv('./PerguntasRe.csv', index = False)